In [1]:
# Set some environment variables
import os
gpu_num = 0 # GPU to be used. Use "" to use the CPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Suppress some TF warnings
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    os.system("pip install sionna")
    import sionna

# Configure GPU
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

# Avoid warnings from TensorFlow
import warnings
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings('ignore')

# Fix the seed for reproducible results
tf.random.set_seed(42)

E0000 00:00:1751208912.142759   80090 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751208912.145454   80090 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751208912.152860   80090 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751208912.152873   80090 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751208912.152874   80090 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751208912.152875   80090 computation_placer.cc:177] computation placer already registered. Please check linka

In [1]:
# Import Sionna RT components
from sionna.rt import load_scene, PlanarArray, Transmitter, Receiver, Camera, PathSolver, RadioMapSolver, subcarrier_frequencies

# For link-level simulations
from sionna.phy import mapping,ofdm,utils
from sionna.phy.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset, AWGN
from sionna.phy.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.phy.utils import compute_ber, ebnodb2no, PlotBER
from sionna.phy.ofdm import KBestDetector, LinearDetector, ResourceGrid
from sionna.phy.mapping import Constellation, Mapper, Demapper
from sionna.phy.fec.polar import PolarEncoder, Polar5GEncoder, PolarSCLDecoder, Polar5GDecoder, PolarSCDecoder
from sionna.phy.fec.ldpc import LDPC5GEncoder, LDPC5GDecoder
from sionna.phy.fec.polar.utils import generate_5g_ranking, generate_rm_code
from sionna.phy.fec.conv import ConvEncoder, ViterbiDecoder, BCJRDecoder
from sionna.phy.fec.turbo import TurboEncoder, TurboDecoder
from sionna.phy.fec.linear import OSDecoder
from sionna.phy.mapping import BinarySource
from sionna.phy.utils.metrics import  count_block_errors
from sionna.phy.mimo import StreamManagement
import mitsuba

import scipy.special as sp
import scipy.stats as stats

# For the implementation of the Keras models
from tensorflow.keras import Model
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

no_preview = False # Toggle to False to use the preview widget
                  # instead of rendering for scene visualization

2025-06-30 18:03:25.777442: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-30 18:03:25.785876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751321005.795342    8917 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751321005.798196    8917 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751321005.806032    8917 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [15]:
#scene = load_scene(sionna.rt.scene.munich, merge_shapes=True) # Merge /home/minhaj/weeks_hallshapes to speed-up computations
scene = load_scene('/home/minhaj/weeks_hall_final/weeks_hall_final.xml', merge_shapes=False)
#scene.objects
#scene.radio_materials

In [16]:
# System parameters
subcarrier_spacing = 30e3 # Hz
num_time_steps = 14 # Total number of ofdm symbols per slot

num_tx = 1 # Number of users
num_rx = 1 # Only one receiver considered
num_tx_ant = 1 # Each user has 4 antennas
num_rx_ant = 1 # The receiver is equipped with 16 antennas

# batch_size for CIR generation
batch_size_cir = 1000

In [17]:
# Configure antenna array for all transmitters
scene.tx_array = PlanarArray(num_rows=1,
                             num_cols=1,
                             vertical_spacing=0,
                             horizontal_spacing=0,
                             pattern="iso",
                             polarization="V")


# Create transmitter
tx = Transmitter(name="tx",
                 position=[-1.4,2.2,1.1],
                 display_radius=0.3)

# Add transmitter instance to scene
scene.add(tx)


bird_cam = Camera(position=[-1.5,0,25], orientation=np.array([0,np.pi/2,-np.pi/2]))
#scene.preview();

In [18]:
# Set the radio map center (elevation control)
center = mitsuba.Point3f(0, 0, 0.2)  # 2.0 meters for the radio map center

# Define the orientation (optional, if you want to tilt the radio map)
orientation = mitsuba.Point3f(0, 0, 0)  # Adjust angles as needed

# Define the size of the radio map (covering the entire scene)
size = mitsuba.Point2f(100, 100)  # Adjust size to focus on the area of interest

# Define cell size (resolution of the radio map)
cell_size = mitsuba.Point2f(0.1, 0.1)  # Size of each cell in the grid

# Set the number of samples per transmitter (more samples = higher accuracy)
samples_per_tx = 10000000

# Set the max depth (for path tracing)
max_depth = 2

# Configure the radio map solver
rm_solver = RadioMapSolver()

# Compute the radio map with the new parameters
rm = rm_solver(scene,
               center=center,
               orientation=orientation,
               size=size,
               cell_size=cell_size,
               samples_per_tx=samples_per_tx,
               max_depth=max_depth)

# Optional: Render the radio map from the bird's eye view
bird_cam = Camera(position=[-1.5, 0, 25], orientation=np.array([0, np.pi/2, -np.pi/2]))

scene.preview(radio_map=rm,
              rm_vmin=-90,
              clip_at=3.); # Clip the scene at rendering for visualizing the refracted field

In [ ]:
import matplotlib as mpl

mpl.rcParams['font.family']       = 'serif'
mpl.rcParams['font.serif']        = ['Times New Roman']
mpl.rcParams['font.size']         = 16

fig = scene.render(
    camera=bird_cam,
    radio_map=rm,
    rm_vmin=-80,
    rm_show_color_bar=True,
    clip_at=3,
    resolution=(1024,768),
    num_samples=512
)

# assume the colorbar was drawn as the last axes in the figure
cbar_ax = fig.axes[-1]
cbar_ax.set_ylabel("Path Gain (dB)", fontsize=18, fontweight="bold")

fig.savefig("radio_map.png", dpi=300, bbox_inches="tight")
plt.show()



In [18]:
max_depth = 3

# Radio map solver
rm_solver = RadioMapSolver()

# Compute the radio map
rm = rm_solver(scene,
               max_depth=5,
               cell_size=(0.1, 0.1),
               samples_per_tx=10**7)

In [19]:
if no_preview:
    # Render an image
    scene.render(camera=bird_cam,
                 radio_map=rm,
                 rm_vmin=-110,
                 clip_at=2.); # Clip the scene at rendering for visualizing the refracted field
else:
    # Show preview
    scene.preview(radio_map=rm,
                  rm_vmin=-110,
                  clip_at=3.); # Clip the scene at rendering for visualizing the refracted field

In [20]:
min_gain_db = -130 # in dB; ignore any position with less than -130 dB path gain
max_gain_db = 0 # in dB; ignore strong paths

# Sample points in a 5-400m range around the receiver
min_dist = 1 # in m
max_dist = 20 # in m

# Sample batch_size random user positions from the radio map
ue_pos, _ = rm.sample_positions(num_pos=batch_size_cir,
                                metric="path_gain",
                                min_val_db=min_gain_db,
                                max_val_db=max_gain_db,
                                min_dist=min_dist,
                                max_dist=max_dist)

In [21]:
# Configure antenna array for all receivers (=UEs)
scene.rx_array = PlanarArray(num_rows=2,
                             num_cols= 2,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="iso",
                             polarization="V")

# Create batch_size receivers
for i in range(batch_size_cir):
    scene.remove(f"rx-{i}") # Remove old receiver if any
    rx = Receiver(name=f"rx-{i}",
                  position=ue_pos[0][i], # Position sampled from radio map
                  velocity=(3.,3.,0),
                  display_radius=1., # optional, radius of the sphere for visualizing the device
                  color=(1,0,0) # optional, color for visualizing the device
                  )
    scene.add(rx)

# And visualize the scene
if no_preview:
    # Render an image
    scene.render(camera=bird_cam,
                 radio_map=rm,
                 rm_vmin=-110,
                 clip_at=2.); # Clip the scene at rendering for visualizing the refracted field
else:
    # Show preview
    scene.preview(radio_map=rm,
                  rm_vmin=-110,
                  clip_at=2.); # Clip the scene at rendering for visualizing the refracted field

In [30]:
target_num_cirs = 500 # Defines how many different CIRs are generated.
# Remark: some path are removed if no path was found for this position

max_depth = 3
min_gain_db = -130 # in dB / ignore any position with less than -130 dB path gain
max_gain_db = 0 # in dB / ignore any position with more than 0 dB path gain

# Sample points within a 10-400m radius around the transmitter
min_dist = 1 # in m
max_dist = 20 # in m

# List of channel impulse reponses
a_list = []
tau_list = []

# Maximum number of paths over all batches of CIRs.
# This is used later to concatenate all CIRs.
max_num_paths = 0

# Path solver
p_solver = PathSolver()

# Each simulation returns batch_size_cir results
num_runs = int(np.ceil(target_num_cirs/batch_size_cir))
for idx in range(num_runs):
    print(f"Progress: {idx+1}/{num_runs}", end="\r")

    # Sample random user positions
    ue_pos, _ = rm.sample_positions(
                        num_pos=batch_size_cir,
                        metric="path_gain",
                        min_val_db=min_gain_db,
                        max_val_db=max_gain_db,
                        min_dist=min_dist,
                        max_dist=max_dist,
                        seed=idx) # Change the seed from one run to the next to avoid sampling the same positions

    # Update all receiver positions
    for rx in range(batch_size_cir):
        scene.receivers[f"rx-{rx}"].position = ue_pos[0][rx]

    # Simulate CIR
    paths = p_solver(scene, max_depth=max_depth, max_num_paths_per_src=10**7)

    # Transform paths into channel impulse responses
    a, tau = paths.cir(sampling_frequency=subcarrier_spacing,
                         num_time_steps=14,
                         out_type='numpy')
    a_list.append(a)
    tau_list.append(tau)

    # Update maximum number of paths over all batches of CIRs
    num_paths = a.shape[-2]
    if num_paths > max_num_paths:
        max_num_paths = num_paths

# Concatenate all the CIRs into a single tensor along the num_rx dimension.
# First, we need to pad the CIRs to ensure they all have the same number of paths.
a = []
tau = []
for a_,tau_ in zip(a_list, tau_list):
    num_paths = a_.shape[-2]
    a.append(np.pad(a_, [[0,0],[0,0],[0,0],[0,0],[0,max_num_paths-num_paths],[0,0]], constant_values=0))
    tau.append(np.pad(tau_, [[0,0],[0,0],[0,max_num_paths-num_paths]], constant_values=0))
a = np.concatenate(a, axis=0) # Concatenate along the num_rx dimension
tau = np.concatenate(tau, axis=0)


print("Shape of a:", a.shape)
print("Shape of tau: ", tau.shape)
# Let's now convert to uplink direction, by switing the receiver and transmitter
# dimensions
a = np.transpose(a, (2,3,0,1,4,5))
tau = np.transpose(tau, (1,0,2))
print("Shape of a:", a.shape)
print("Shape of tau: ", tau.shape)
# Add a batch_size dimension
a = np.expand_dims(a, axis=0)
tau = np.expand_dims(tau, axis=0)

# Exchange the num_tx and batchsize dimensions
a = np.transpose(a, [3, 1, 2, 0, 4, 5, 6])
tau = np.transpose(tau, [2, 1, 0, 3])

# Remove CIRs that have no active link (i.e., a is all-zero)
p_link = np.sum(np.abs(a)**2, axis=(1,2,3,4,5,6))
a = a[p_link>0.,...]
tau = tau[p_link>0.,...]

print("Shape of a:", a.shape)
print("Shape of tau: ", tau.shape)

Shape of a: (1000, 4, 1, 1, 18, 14)
Shape of tau:  (1000, 1, 18)
Shape of a: (1, 1, 1000, 4, 18, 14)
Shape of tau:  (1, 1000, 18)
Shape of a: (874, 1, 1, 1, 4, 18, 14)
Shape of tau:  (874, 1, 1, 18)


In [12]:
target_num_cirs = 5000 # Defines how many different CIRs are generated.
# Remark: some path are removed if no path was found for this position

max_depth = 5
min_gain_db = -130 # in dB / ignore any position with less than -130 dB path gain
max_gain_db = 0 # in dB / ignore any position with more than 0 dB path gain

# Sample points within a 10-400m radius around the transmitter
min_dist = 10 # in m
max_dist = 40 # in m

# List of channel impulse reponses
a_list = []
tau_list = []

# Maximum number of paths over all batches of CIRs.
# This is used later to concatenate all CIRs.
max_num_paths = 0

# Path solver
p_solver = PathSolver()

In [13]:
batch_size = 20 # Must be the same for the BER simulations as CIRDataset returns fixed batch_size

# Init CIR generator
cir_generator = CIRGenerator(a,
                             tau,
                             num_tx)
# Initialises a channel model that can be directly used by OFDMChannel layer
channel_model = CIRDataset(cir_generator,
                           batch_size,
                           num_rx,
                           num_rx_ant,
                           num_tx,
                           num_tx_ant,
                           max_num_paths,
                           num_time_steps)

I0000 00:00:1750176746.213114    6905 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4560 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
